In [88]:
from tensorflow.keras import models, layers, activations, \
    optimizers, utils, losses, initializers, metrics, callbacks

In [89]:
epochs = 100
batch_size = 32
patience = 3
learning_rate = 1e-4
model_path = './model/sigmoid.keras'

In [90]:
model = models.Sequential([
    layers.Resizing(128,128),
    layers.Rescaling(1.0/255),
    layers.BatchNormalization(),

    layers.Conv2D(32,(5,5),
        activation='relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(64,(3,3),
        activation='relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),


    layers.Conv2D(128,(3,3),
        activation='relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(64,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(32,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(5,
        activation = 'softmax',
        kernel_initializer = initializers.RandomNormal()
    )
])

In [91]:
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = learning_rate
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(name="accuracy")]
)

In [92]:
train = utils.image_dataset_from_directory(
    'Data',
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    shuffle = True,
    image_size = (512, 512),
    batch_size = 32
)
test =  utils.image_dataset_from_directory(
    'Data',
    validation_split = 0.2,
    subset = 'validation',
    seed = 123,
    shuffle = True,
    image_size = (512, 512),
    batch_size = 32
)
train.class_names


Found 3276 files belonging to 5 classes.
Using 2621 files for training.
Found 3276 files belonging to 5 classes.
Using 655 files for validation.


['hammer', 'nothing', 'pliers', 'screwdriver', 'wrench']

In [93]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 40s 395ms/step - accuracy: 0.3273 - loss: 1.4646 - val_accuracy: 0.3557 - val_loss: 1.4762
Epoch 2/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 33s 401ms/step - accuracy: 0.4492 - loss: 1.1488 - val_accuracy: 0.3420 - val_loss: 1.3972
Epoch 3/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 33s 407ms/step - accuracy: 0.4955 - loss: 1.0909 - val_accuracy: 0.3588 - val_loss: 1.3197
Epoch 4/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 31s 372ms/step - accuracy: 0.5267 - loss: 1.0413 - val_accuracy: 0.4489 - val_loss: 1.1256
Epoch 5/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 34s 411ms/step - accuracy: 0.5645 - loss: 0.9853 - val_accuracy: 0.4779 - val_loss: 1.1175
Epoch 6/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 32s 384ms/step - accuracy: 0.5934 - loss: 0.9657 - val_accuracy: 0.5420 - val_loss: 1.0360
Epoch 7/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 33s 407ms/step - accuracy: 0.6155 - loss: 0.9074 - val_accuracy: 0.5710 - val_loss: 0.9884
Epoch 8/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 38s 462ms/step - accuracy: 0.6506 - loss: 0.8425 - 

In [94]:
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [95]:
model = load_model(model_path)

In [96]:
def load_img(img_path):
    img = Image.open(img_path)
    img_array = np.asarray(img)
    return np.expand_dims(img_array, axis=0)

In [104]:
e = []
e.append(np.argmax(model.predict(load_img("data/hammer/000003.jpg"))))
e.append(np.argmax(model.predict(load_img("data/nothing/nothing1.jpg"))))
e.append(np.argmax(model.predict(load_img("data/Pliers/000000.jpg"))))
e.append(np.argmax(model.predict(load_img("data/screwdriver/000000.jpg"))))
e.append(np.argmax(model.predict(load_img("data/wrench/000000.jpg"))))

print(e)
np.argmax(model.predict(load_img("3.jpeg")))



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


np.int64(3)